In [33]:
import pandas as pd
null="\u03bb"

In [50]:
# reading FA from file
inputFile="F:/Coding projects/FiniteAutomata/FA CSV input files/DFA10.csv"
FA = pd.read_csv(inputFile, index_col="states", dtype="str") 
FA.index = FA.index.map(str)
print(FA)


# convert data to appropriate type (using frozen set for states as they are hashable and can be used as index)
FA.isInitial = FA["isInitial"].map({'FALSE':False, 'TRUE':True}) # bool
#NFA.index = [frozenset(x) for x in NFA.index] # bool
FA.isFinal = FA["isFinal"].map({'FALSE':False, 'TRUE':True}) # bool

# find final states, intial state  and symbols 
q0= FA.loc[FA["isInitial"]==True].index.values[0]
print("inital Stated of NFA=",q0)
finalStates=set(FA.loc[FA["isFinal"]==True].index.values.tolist())
print("final states: ", finalStates)
symbols= [x for x in list(FA.columns) if x.find("input")!=-1]
print("symbols: ", symbols)
states= FA.index.values.tolist()
print("States", states)



       input_a input_b isInitial isFinal
states                                  
A            B       B      TRUE    TRUE
B            C       A     FALSE    TRUE
C            C       A     FALSE   FALSE
inital Stated of NFA= A
final states:  {'A', 'B'}
symbols:  ['input_a', 'input_b']
States ['A', 'B', 'C']


In [51]:
# create table to store transitions between each pair of state
# also add a new initial and new final state
table= pd.DataFrame(index=states+["NI","NF"], columns=states+["NI","NF"])

# add tran(NI,null)=q0 and tran(finalstates, null)=NF
table.loc["NI",q0]= null
table.loc[finalStates,"NF"]= null

for source in states:
    for s in symbols:
        if FA[s][source]==FA[s][source]:
            destinations= FA[s][source].split(",")
            for dest in destinations:
                re=table[dest][source] 
                if re==re:
                    re=re+" + "+s.replace("input_","")
                else: 
                    re=s.replace("input_","")
                table[dest][source] =re
        
print(table)                    

      A      B    C   NI   NF
A   NaN  a + b  NaN  NaN    λ
B     b    NaN    a  NaN    λ
C     b    NaN    a  NaN  NaN
NI    λ    NaN  NaN  NaN  NaN
NF  NaN    NaN  NaN  NaN  NaN


In [52]:

# eliminate states one by one
# states will be elimiated in order of their occurance in DFA index
for x in states:
    print("Eliminating state ",x, "\n________________________________")
    #print(table)
    loop= table.loc[x,x]  if table.loc[x,x]==table.loc[x,x] else ""    
    loop= loop+"*" if len(loop)==1 else ("" if len(loop)==0 else"("+loop+")*")
    outward=[y for y in table.loc[(table.loc[x].isna()==False)].index.values if y!=x]
    inward=[y for y in table[table[x].isna()==False].index.values if y!=x]
    print("inward transitions:",inward)
    print("loop on state", x,":", (loop[0:-1] if loop!="" else null))    
    print("outward transitions:",outward, "\n")
    
    
    for i in inward:
        for o in outward:
            print("-"  ,i, "-->", o)
            
            
            
            REin= (table.loc[i,x] if table.loc[i,x]==table.loc[i,x] else "")
            REout=(table.loc[x,o] if table.loc[x,o]==table.loc[x,o] else "")
            REold=(table.loc[i,o] if table.loc[i,o]==table.loc[i,o] else "")
            
            print(
                  "\n  RE from ", i , "to", x, "is ",(REin if REin!="" else "-NA-"),
                  "\n  RE of loop is ", (loop if loop!="" else null) , 
                  "\n  RE from ", x , "to", o, "is ",(REout if REout!="" else "-NA-"),
                  "\n  old RE from ", i , "to", o, "is ",(REold if REold!="" else "-NA-"),
                
            )
            if (REin==null and REout!=""):
                REin=""
            if (REout==null and REin!=""):
                REout=""   
                
            if "+" in REin and loop+REout!="":
                REin= "("+REin+")"
            if "+" in REout and loop+REin!="":
                REout= "("+REout+")"
                
            
            
            new_RE= REin + loop + REout
            if REold!="":
                new_RE= new_RE+" + "+REold
                #new_RE= "("+new_RE+" + "+REold+")"
            
            print("\n  Therefore ",i,"-->", o, "through", x, "is", new_RE ,"\n",) 
            
            
            table.loc[i,o]= new_RE
            #table.loc[i,o]=" ( "+(table.loc[i,x]+loop+table.loc[x,o])+ (" + "+table.loc[i,o] if table.loc[i,o]==table.loc[i,o] else "")+" ) "
        print("\n\n")
    table = table.drop(x, 1)
    table = table.drop(x, 0)
    #print(table) 

Eliminating state  A 
________________________________
inward transitions: ['B', 'C', 'NI']
loop on state A : λ
outward transitions: ['B', 'NF'] 

- B --> B

  RE from  B to A is  b 
  RE of loop is  λ 
  RE from  A to B is  a + b 
  old RE from  B to B is  -NA-

  Therefore  B --> B through A is b(a + b) 

- B --> NF

  RE from  B to A is  b 
  RE of loop is  λ 
  RE from  A to NF is  λ 
  old RE from  B to NF is  λ

  Therefore  B --> NF through A is b + λ 




- C --> B

  RE from  C to A is  b 
  RE of loop is  λ 
  RE from  A to B is  a + b 
  old RE from  C to B is  -NA-

  Therefore  C --> B through A is b(a + b) 

- C --> NF

  RE from  C to A is  b 
  RE of loop is  λ 
  RE from  A to NF is  λ 
  old RE from  C to NF is  -NA-

  Therefore  C --> NF through A is b 




- NI --> B

  RE from  NI to A is  λ 
  RE of loop is  λ 
  RE from  A to B is  a + b 
  old RE from  NI to B is  -NA-

  Therefore  NI --> B through A is a + b 

- NI --> NF

  RE from  NI to A is  λ 
  RE of lo

In [29]:
print("Regular expression for the given FA is \n",table.loc["NI","NF"])
print("Note the the this is only one possible RE, results will be different if states are elimimated in different")

Regular expression for the given FA is 
 ((aa*a + b)(ba*a)*ba*(a + b) + aa*(a + b))
Note the the this is only one possible RE, results will be different if states are elimimated in different
